In [0]:
# utils notebook

def upsert_table(source_table: str, target_table: str, merge_keys: list):
    """
    Perform an upsert (MERGE) from source_table into target_table.
    """
    merge_condition = " AND ".join([f"target.{k} = source.{k}" for k in merge_keys])
    
    # Ensure target exists (empty schema copy if new)
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {target_table}
    USING DELTA
    AS SELECT * FROM {source_table} WHERE 1=0
    """)
    
    merge_sql = f"""
    MERGE INTO {target_table} AS target
    USING {source_table} AS source
    ON {merge_condition}
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
    """
    
    print(f"MERGE {source_table} → {target_table} ON {merge_keys}")
    spark.sql(merge_sql)
